## Imports and Data

Include pytorch

In [0]:
import torch

Include and install Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git
import os
os.chdir("fairseq/")
!pip install -r requirements.txt
%run -i 'setup.py' build develop

Cloning into 'fairseq'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 3417 (delta 12), reused 7 (delta 7), pack-reused 3392
Receiving objects: 100% (3417/3417), 3.31 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (2482/2482), done.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/tests
copying tests/test_train.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_average_checkpoints.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_noising.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_backtranslation_dataset.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_sequence_generator.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_reproducibility.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_token_block_dataset.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_d

Ukuxhumana repo includes the data needed. Subword provides scripts to do bpe.

In [8]:
!git clone https://github.com/LauraMartinus/ukuxhumana.git
!git clone https://github.com/rsennrich/subword-nmt

fatal: destination path 'ukuxhumana' already exists and is not an empty directory.
fatal: destination path 'subword-nmt' already exists and is not an empty directory.


## Training of Various Models

### Standard Transformer
Using clean data, BPE for 4000.

In [0]:
os.chdir('../')
!cat fairseq/ukuxhumana/clean/en_zu/enzu_parallel.train.en fairseq/ukuxhumana/clean/en_zu/enzu_parallel.train.zu > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 4000 <combine.txt> enzu.codes

!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.train.zu > train.zu
!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.dev.zu > valid.zu
!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < fairseq/ukuxhumana/clean/en_zu/enzu_parallel.test.zu > test.zu

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang zu --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enzu

!mkdir -p chckpoint/transformer
!python train.py data-bin/enzu \
  -a transformer_iwslt_de_en --optimizer adam --lr 0.0005 -s en -t zu \
  --label-smoothing 0.1 --dropout 0.3 --max-tokens 4000 \
  --min-lr '1e-09' --lr-scheduler inverse_sqrt --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --max-update 50000 \
  --warmup-updates 4000 --warmup-init-lr '1e-07' \
  --adam-betas '(0.9, 0.98)' --save-dir checkpoints/transformer

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enzu --path checkpoints/transformer/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang zu

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='enzu.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Pleas

In [0]:
# os.chdir('fairseq/')
# TEXT=".."
# %run 'preprocess.py' --source-lang en --target-lang zu --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enzu

!mkdir -p chckpoint/transformer
!python train.py data-bin/enzu --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 200 --arch transformer --save-dir chckpoint/transformr 
    #--lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    #--arch transformer --save-dir checkpoint/transformer

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enzu --path chckpoint/transformr/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang zu

Namespace(adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer', attention_dropout=0.0, bucket_cap_mb=25, clip_norm=0.1, cpu=False, criterion='cross_entropy', data=['data-bin/enzu'], ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, distributed_backend='nccl', distributed_init_method=None, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, encoder_attention_heads=8, encoder_embed_dim=512, encoder_embed_path=None, encoder_ffn_embed_dim=2048, encoder_layers=6, encoder_learned_pos=False, encoder_normalize_before=False, fix_batches_to_gpus=False, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, keep_interval_updates=-1, keep_last_epochs=-1, lazy_load=False, left_pad_source='True', left_pad_target='F

OSError: ignored

In [0]:
!ls

In [0]:
# Average 10 latest checkpoints:
!python scripts/average_checkpoints.py --inputs checkpoint/fconv/ \
   --num-epoch-checkpoints 4 --output checkpoint/fconv/model.pt

Namespace(checkpoint_upper_bound=None, inputs=['checkpoint/fconv/'], num_epoch_checkpoints=4, num_update_checkpoints=None, output='checkpoint/fconv/model.pt')
averaging checkpoints:  ['checkpoint/fconv/checkpoint8.pt', 'checkpoint/fconv/checkpoint7.pt', 'checkpoint/fconv/checkpoint6.pt', 'checkpoint/fconv/checkpoint5.pt']
Finished writing averaged checkpoint to checkpoint/fconv/model.pt.


In [0]:
!python generate.py data-bin/enzu \
  --path checkpoint/fconv/model.pt \
  --batch-size 128 --beam 5 --remove-bpe

Namespace(beam=5, cpu=False, data=['data-bin/enzu'], diverse_beam_groups=-1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, memory_efficient_fp16=False, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, no_repeat_ngram_size=0, num_shards=1, num_workers=0, path='checkpoint/fconv/model.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang=None, target_lang=None, task='translation', unkpen=0, unn

**RESULT**


### Multilingual
Using Transformer, training and validating with multilingual data, and then testing with just zulu data.
Should run on various values for bpe. Start with 32 000 just because that is what the Johnson paper does.

In [4]:
os.chdir('../')
!cat fairseq/ukuxhumana/multilingual/all_parallel.train.en fairseq/ukuxhumana/multilingual/all_parallel.train.all > combine.txt
!fairseq/subword-nmt/learn_bpe.py -s 32000 <combine.txt> enall.codes

!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/all_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/all_parallel.train.all > train.zu
!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/all_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/all_parallel.dev.all > valid.zu
!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/enzu_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enall.codes < fairseq/ukuxhumana/multilingual/enzu_parallel.test.zu > test.zu

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang zu --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enzu

!mkdir -p checkpoint/fconv
!python train.py data-bin/enzu \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enzu --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang zu

fairseq/subword-nmt/learn_bpe.py:267: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Please point to the new location, or install the package and use the command 'subword-nmt'
  DeprecationWarning
fairseq/subword-nmt/apply_bpe.py:351: ResourceWarning: unclosed file <_io.TextIOWrapper name='enall.codes' mode='r' encoding='UTF-8'>
  args.codes = codecs.open(args.codes.name, encoding='utf-8')
fairseq/subword-nmt/apply_bpe.py:334: DeprecationWarning: this script's location has moved to /content/fairseq/subword-nmt/subword_nmt. This symbolic link will be removed in a future version. Plea

IndexError: ignored

In [7]:
output = %run 'generate.py' data-bin/enzu --path checkpoint/fconv/checkpoint7.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang zu

Namespace(beam=5, cpu=False, data=['data-bin/enzu'], diverse_beam_groups=-1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, memory_efficient_fp16=False, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, no_repeat_ngram_size=0, num_shards=1, num_workers=0, path='checkpoint/fconv/checkpoint7.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sacrebleu=False, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang='en', target_lang='zu', task='tran

IndexError: ignored

**RESULT**

### Using Morfessor
